# You Only Look Once: Unified, Real-Time Object Detection  
- paper review

## Abstract  
- 우리는 object detection을 bounding box와 이의 class probability를 regression하는 문제로 만든다  
- YOLO의 pipeline은 single network이며 end-to-end로 학습한다  
- base YOLO model은 초당 45 frame의 속도를 보여주며 Fast YOLO는 155 frame의 속도를 보여준다  
- detection SOTA와 비교해 YOLO는 localization error가 많지만 background에서 false positive를 덜 예측한다  
- 이는 DPM, R-CNN 등 다른 모델들의 성능을 능가한다  

## Introduction  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/figure1.png?raw=true" width=600>  


- R-CNN과 같은 모델들은 potential bounding box를 예측하기 위해 region-proposal method를 사용한다  
- 그리고 나서 NMS 등의 post-preprocess 작업을 거친다  
- 또한 단점으로 느린 inference speed와 어려운 optimization이 있다  
- 그래서 우리는 object detection을 single regression problem으로 재구성한다  
- YOLO는 하나의 CNN으로 bounding box와 class probability를 동시에 예측한다  
- 먼저 YOLO는 굉장히 빠르며 복잡하지 않은 pipeline을 가지고 있다  
- 또한 다른 real-time system보다 2배 높은 mean AP를 보여준다  
- 두 번째로, YOLO는 이미지 전체에 대해서 추론을 한다  
- sliding window, region proposal-based techniques와 달리 YOLO는 전체 이미지를 살펴보며 이미지의 모양 뿐만 아니라 class에 대한 contextual information을 encoding한다  
- Faster R-CNN 모델의 경우, context information을 얻지 못해 background patch를 object로 착각하는 경우가 많다  
- YOLO는 Faster R-CNN보다 더 적은 background error를 가진다  
- 세 번째로, YOLO는 object의 일반화할 수 있는 representation을 학습한다  
- 이는 새로운 domain이나 unexpected image에 대해서도 일반화된 성능을 보여줌을 의미한다  
- 다만 단점으로 작은 object에 대해서는 정확하게 localize하지 못하는 경향이 있다

## Unified Detection  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/figure2.png?raw=true" width=500>  

- YOLO는 real-time 그리고 end-to-end network임에도 불구하고 높은 AP값을 보여준다  
- 우리 system은 input image를 $S\times S$ grid로 나눈다  
- 만약 object의 center가 grid cell에 속한다면 이 grid cell은 object를 탐지하는 역할을 한다  
- 우리는 **confidence**를 $Pr\left(Object\right) * IOU_{pred}^{truth}$로 정의한다  
- 만약 object가 cell에 존재하지 않는다면 confidence score는 0이 되어야 한다  
- 그렇지 않다면 cofidence score는 IOU와 동일해지길 원한다  
- 각 bounding box는 5개의 정보를 담고 있다: $x$, $y$, $w$, $h$ 그리고 confidence. 
- 또한 각 grid cell은 $C$ conditional class probability $Pr\left(Class_i\vert Object\right)$를 예측한다  
- 우리는 box의 갯수에 상관없이 grid cell 당 class probability의 one set을 예측한다   
- test time에 우리는 conditional class probability와 각 box confidence를 다음과 같이 곱한다  
$$Pr\left(Class_i\vert Object\right) * Pr\left(Object\right) * IOU_{pred}^{truth} = Pr\left(Class_i\right) * IOU_{pred}^{truth}$$  

### Network Design  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/figure3.png?raw=true">

- network의 convolutional layer는 image의 feature를 추출하며 fc layer는 probability와 coordinate를 예측한다  
- YOLO는 24개의 convolutional layer, 2개의 fc layer로 이루어져 있다  
- 그리고 $1\times 1$ conv layer 다음에 $3\times 3$ conv layer로 구성을 했다  
- final output은 $7\times 7\times 30$ tensor로 이루어져 있다  

### Training  
- YOLO의 conv layer는 ImageNet-1K dataset으로 pretrain을 진행했다  
- 작은 object를 탐지하기 위해 image size를 $224\times 224$부터 $448\times 448$로 점차 늘려갔다  
- output layer는 class probability와 box coordinate를 예측하는데 이때 0~1 normalizaed된 값을 사용한다  
- finaly layer에는 linear activation function을 사용하며 그 외에는 leaky ReLU activation function을 사용한다  
- 우리는 sum-squared error를 최적화를 한다  
- 이는 최적화하기 편리하지만 AP를 maximizing 하기에는 어려움이 있다  
- 보통 이미지에서 대부분의 grid cell은 object를 포함하지 않는다  
- 이는 no object의 gradient가 object의 gradient를 압도해버리는 문제가 생긴다  
- 이를 개선하기 위해 box coordinate의 weight는 증가시키고 confidence의 weight는 감소시킨다   
- 즉, $\lambda_{coord}=5$, $\lambda_{noobj}=0.5$  
- SSE에서 large box와 small box에 대한 error의 weight는 동일하다  
- 그래서 small box보다 large box의 편차가 덜 중요하다는 것을 loss function에 반영해야 한다  
- 이를 부분적으로 해결하기 위해 directly width, height 대신 square root를 씌운 값을 예측한다  
- model의 loss function은 다음과 같다  
$$\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(x_i-\hat{x}_i\right)^2 + \left(y_i-\hat{y}_i\right)^2\right]$$  
$$+\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(\sqrt{w_i}-\sqrt{\hat{w}_i}\right)^2 + \left(\sqrt{h_i}-\sqrt{\hat{h}_i}\right)^2\right]$$  
$$+\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left(C_i-\hat{C}_i\right)^2$$  
$$+\lambda_{noobj}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{noobj}\left(C_i-\hat{C}_i\right)^2$$  
$$+\sum_{i=0}^{S^2}\mathcal{1}_{i}^{obj}\sum_{c\in classes}\left(p_i\left(c\right)-\hat{p}_i\left(c\right)\right)$$  
- 이때 $\mathcal{1}_i^{obj}$는 $i$번째 cell에 object가 존재하는지, $\mathcal{1}_{ij}^{obj}$는 $i$번째 cell, $j$번째 box에 object가 존재하는지 나타낸다  
- hyperparameter로 batch size는 64, momentum 0.9 그리고 decay는 0.0005를 사용한다  
- learning rate schedule로 처음 75 epoch까지는 $10^{-2}$로 학습하고 그 이후 30 epoch까지는 $10^{-3}$, 마지막 30 epoch 동안 $10^{-4}$로 학습한다  
- overfitting을 피하기 위해 우리는 dropout과 data augmentation을 적용했다  
- dropout rate는 0.5를 사용하며 first fc layer 이후에 적용한다  
- data augmentation으로 random scaling with scale 1.2, random HSV color space를 적용한다  

### Inference  
- YOLO는 PASCAL VOC에서 이미지당 98개의 box와 class probability를 예측한다  
- grid design은 bounding box prediction의 다양성을 보여준다  
- 그러나 몇몇 large object 또는 grid cell 경계 근처 object는 multiple cell에 의해 localize가 잘 될 수 있다  

### Limitations of YOLO  
- YOLO는 grid cell 당 2개의 box밖에 예측하지 못해 spatial constraint가 존재한다  
- 이는 작은 object나 한 grid cell 안에 여러 개의 object가 존재하는 경우 모델의 성능이 떨어지게 만든다  
- YOLO는 data로부터 학습을 하기 때문에 새롭거나 특이한 aspect ratio 또는 환경설정에서 object detection 성능을 일반화하기 어렵다  
- 마지막으로 loss function을 최적화하는 동안 small box에 대한 error와 large box에 대한 error를 똑같이 취급한다  
- large box의 small error는 일반화가 잘 되는 반면 small box의 small error는 IOU에 훨씬 더 큰 영향을 미친다  
- 이 원인은 잘못된 localization이다  

## Comparison to Other Detection Systems  
**Deformable parts models**  
- DPM은 static feature를 추출하고 region을 분류하고 bounding box를 예측하기 위해 분리된 pipeline을 사용한다  
- 우리 system은 각 part를 하나의 convolutional neural network로 대체했다  
- 하나의 network 안에 feature extraction, bounding box prediction, non-maximal suppression, contexual reasoning이 들어있다  

**R-CNN**  
- R-CNN과 변형 모델들은 sliding window 대신 region proposal을 사용한다  
- end-to-end가 아니다 보니 각 part는 정확하게 수행되어야 한다  
- 그래서 image 당 40초가 걸리는 매우 느린 속도를 가지고 있다  
- YOLO는 R-CNN과 비슷한 점을 지니고 있다  
- grid cell에서 potential bounding box를 예측하고 이들을 convolutional feature를 이용해 score를 매긴다  
- 그러나 우리 시스템은 각 grid cell당 2개의 box만을 예측한다  
- Selective Search가 이미지당 약 2000개의 boundig box를 제안하는 것과 달리 YOLO는 98개만 제안한다  

## Experiments  
- YOLO를 PASCAL VOC 2007 dataset에서 다른 모델들과 성능을 비교한다  

### Comparison to Other Real-Time Systems  
<img src  = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/table1.png?raw=true" width=600>  

- real-time detector로 YOLO는 가장 높은 mAP 값을 가지고 있으며 FPS도 real-time으로 충분히 빠른 속도를 가진다  

### VOC 2007 Error Analysis  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/figure4.png?raw=true" width=600>  

- Fast R-CNN과 비교해 YOLO는 localization error가 더 높은 수치를 보이고 있다  
- 그러나 background error가 훨씬 더 적은 모습을 보여준다  

### Combining Fast R-CNN and YOLO  
- YOLO와 Fast R-CNN을 결합하면 어떤 성능을 내는지 살펴보자  

<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/table2.png?raw=true" width=550>  

- YOLO와 Fast R-CNN을 결합하면 조금 더 좋은 성능을 보여주는 것을 알 수 있다  

### VOC 2012 Results  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/table3.png?raw=true">  

- VOC 2012 test set에서 YOLO는 57.9%의 mAP를 달성했다  

### Generalizability: Person Detection in Artwork  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/figure5.png?raw=true" width=700>  

## Real-Time Detection In The Wild  
<img src = "https://github.com/Sangh0/Object-Detection/blob/main/YOLO/figure/figure6.png?raw=true">  


## Conclusion  
- 우리는 통합된 네트워크 YOLO를 소개했으며 매우 간단하고 빠르게 동작되는 모델이다  
- 또한 YOLO는 새롭거나 unexpected한 이미지에 대해서도 좋은 성능을 보여준다  
- 즉, 다양한 application에서도 활용할 가치가 충분하다